In [1]:
import glob
import os
from datetime import datetime as dt

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from helpers import build_season_mv


In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)

In [3]:
data_path = '/Users/calumthompson/Documents/Fantasy_football/GitHub_data/data/'
season_folders = glob.glob(data_path + '*')
season_names = [x.replace(data_path, '') for x in season_folders]

# Summary

## Features 

- Player
- Position
- Team
- Opposition
- Home/away
- Gameweek
- Season
- Minutes
- Team market value at start of season (relative to all teams)
- Opposition market value at start of season (relative to all teams)

## Target

- Total_points

# Update github data

In [4]:
github_data_address = "/Users/calumthompson/Documents/Fantasy_football/GitHub_data/"
working_folder = os.getcwd() + '/'

print("Access data folder")
%cd $github_data_address
print("")

print("Git Pull....")
! git pull
print("")

print("Data last updated: ")
%cd "/Users/calumthompson/Documents/Fantasy_football/GitHub_data/data"
! git show --no-patch --no-notes --pretty='%cd'

print("")
print("Returning to original directory")
%cd $working_folder

Access data folder
/Users/calumthompson/Documents/Fantasy_football/GitHub_data

Git Pull....
Already up to date.

Data last updated: 
/Users/calumthompson/Documents/Fantasy_football/GitHub_data/data
Mon May 18 02:06:49 2020 +0000

Returning to original directory
/Users/calumthompson/Documents/Fantasy_football/Cals-FF-model/Soloman method


# Get fixtures for each year

## From fixture table

In [5]:
fixtures_link = {'2019-20' : 'https://fixturedownload.com/download/epl-2019-GMTStandardTime.csv'
                ,'2018-19' : 'https://fixturedownload.com/download/epl-2018-GMTStandardTime.csv'
                ,'2017-18' : 'https://fixturedownload.com/download/epl-2017-GMTStandardTime.csv'
                ,'2016-17' : 'https://fixturedownload.com/download/epl-2016-GMTStandardTime.csv'}

In [6]:
Date_format = lambda x: pd.datetime.strptime(x, "%d/%m/%Y %H:%M")

data_merge = []

for i in fixtures_link:
    
    link = fixtures_link[i]
    
    import_df = pd.read_csv(link,parse_dates = ['Date'], date_parser = Date_format)
    
    import_df['Season'] = i
    
    data_merge.append(import_df)
    
    
fixtures_RAW = pd.concat(data_merge)
fixtures_RAW[['home_score' , 'away_score']] = fixtures_RAW['Result'].str.split(' - ', expand = True)

fixtures_RAW['home_score'] = pd.to_numeric(fixtures_RAW['home_score'])
fixtures_RAW['away_score'] = pd.to_numeric(fixtures_RAW['away_score'])

fixtures_RAW.sort_values('Date').tail()

,Round Number,Date,Location,Home Team,Away Team,Result,Season,home_score,away_score
371,38,2020-07-26 15:00:00,Turf Moor,Burnley,Brighton,NaN,2019-20,NaN,NaN
370,38,2020-07-26 15:00:00,Emirates Stadium,Arsenal,Watford,NaN,2019-20,NaN,NaN
378,38,2020-07-26 15:00:00,St. Mary's Stadium,Southampton,Sheffield Utd,NaN,2019-20,NaN,NaN
372,38,2020-07-26 15:00:00,Stamford Bridge,Chelsea,Wolves,NaN,2019-20,NaN,NaN
377,38,2020-07-26 15:00:00,St. James' Park,Newcastle,Liverpool,NaN,2019-20,NaN,NaN


In [7]:
merge = []

# Split into individual seasons
for season in fixtures_RAW['Season'].unique():
    
    season_df = fixtures_RAW.loc[fixtures_RAW['Season'] == season]
    

    for team in season_df['Home Team'].unique():
        
        team_record = season_df.loc[(season_df['Home Team'] == team) | (season_df['Away Team'] == team)]

        col_team = np.full(38, team)
        col_GWs = np.arange(1,39)
        col_date = team_record['Date'].dt.date
        col_Opponent = np.where(team_record['Home Team'] == team, team_record['Away Team'],team_record['Home Team'])
        col_home = np.where((team_record['Home Team'] == team), 1 , 0)
        col_goals_marg = np.where((team_record['Home Team'] == team), team_record['home_score'], team_record['away_score'])
        col_goals_cum  = np.cumsum(col_goals_marg)
        col_opp_goals = np.where((team_record['Home Team'] == team), team_record['away_score'], team_record['home_score'])
        col_points_marg = np.where(col_goals_marg > col_opp_goals, 3, 
                                     np.where(col_goals_marg == col_opp_goals, 1, col_goals_marg))
        col_points_cum = np.cumsum(col_points_marg)

        
        team_record = pd.DataFrame({'FIX_season' : season
                                   ,'FIX_Team':col_team
                                   ,'FIX_GW':col_GWs
                                   ,'FIX_Fixture_date':col_date
                                   ,'FIX_Opponent':col_Opponent
                                   ,'FIX_Home?':col_home
                                   ,'FIX_goals_marginal' : col_goals_marg
                                   ,'FIX_goals_cumulative':col_goals_cum
                                   ,'FIX_opp_goals' : col_opp_goals
                                   ,'FIX_points_marginal' : col_points_marg
                                   ,'FIX_points_cumulative' : col_points_cum })

        merge.append(team_record)
    
fixtures_df = pd.concat(merge).reset_index(drop = True)    


/Users/calumthompson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in greater


In [8]:
fixtures_df.head()

,FIX_season,FIX_Team,FIX_GW,FIX_Fixture_date,FIX_Opponent,FIX_Home?,FIX_goals_marginal,FIX_goals_cumulative,FIX_opp_goals,FIX_points_marginal,FIX_points_cumulative
0,2019-20,Liverpool,1,2019-08-09,Norwich,1,4.0,4.0,1.0,3.0,3.0
1,2019-20,Liverpool,2,2019-08-17,Southampton,0,2.0,6.0,1.0,3.0,6.0
2,2019-20,Liverpool,3,2019-08-24,Arsenal,1,3.0,9.0,1.0,3.0,9.0
3,2019-20,Liverpool,4,2019-08-31,Burnley,0,3.0,12.0,0.0,3.0,12.0
4,2019-20,Liverpool,5,2019-09-14,Newcastle,1,3.0,15.0,1.0,3.0,15.0


## Map teams to Vaastav IDs

In [9]:
team_ids = pd.read_csv('/Users/calumthompson/Documents/Fantasy_football/Cals-FF-model/Soloman method/team_ids.csv'
                      ,usecols = ['team','team_1617','team_1718','team_1819','team_1920']  )

team_ids = team_ids.rename(columns = {'team_1617': '2016-17',
                                      'team_1718': '2017-18',
                                      'team_1819': '2018-19',
                                      'team_1920': '2019-20'})

team_ids = team_ids.melt(id_vars = 'team',
                         var_name = 'FIX_season',
                         value_name = 'FIX_team_code')

team_ids['FIX_team_code'] = pd.to_numeric(team_ids['FIX_team_code'])

team_ids = team_ids.rename(columns = {'team' : 'FIX_team'})

team_ids.loc[team_ids['FIX_team'] == 'West Ham United', 'FIX_team'] = 'West Ham'
team_ids.loc[team_ids['FIX_team'] == 'Tottenham Hotspur', 'FIX_team'] = 'Spurs'
team_ids.loc[team_ids['FIX_team'] == 'Leicester City', 'FIX_team'] = 'Leicester'
team_ids.loc[team_ids['FIX_team'] == 'Newcastle United', 'FIX_team'] = 'Newcastle'
team_ids.loc[team_ids['FIX_team'] == 'Manchester City', 'FIX_team'] = 'Man City'
team_ids.loc[team_ids['FIX_team'] == 'Manchester United', 'FIX_team'] = 'Man Utd'
team_ids.loc[team_ids['FIX_team'] == 'Brighton and Hove Albion', 'FIX_team'] = 'Brighton'
team_ids.loc[team_ids['FIX_team'] == 'Sheffield United', 'FIX_team'] = 'Sheffield Utd'
team_ids.loc[team_ids['FIX_team'] == 'Wolverhampton Wanderers', 'FIX_team'] = 'Wolves'

team_ids.loc[team_ids['FIX_team'] == 'Huddersfield Town', 'FIX_team'] = 'Huddersfield'
team_ids.loc[team_ids['FIX_team'] == 'Cardiff City', 'FIX_team'] = 'Cardiff'
team_ids.loc[team_ids['FIX_team'] == 'West Bromwich Albion', 'FIX_team'] = 'West Brom'
team_ids.loc[team_ids['FIX_team'] == 'Swansea City', 'FIX_team'] = 'Swansea'

team_ids.loc[team_ids['FIX_team'] == 'Stoke City', 'FIX_team'] = 'Stoke'
team_ids.loc[team_ids['FIX_team'] == 'Hull City', 'FIX_team'] = 'Hull'

team_ids.sort_values(['FIX_team', 'FIX_season']).head()

,FIX_team,FIX_season,FIX_team_code
0,Arsenal,2016-17,1.0
29,Arsenal,2017-18,1.0
58,Arsenal,2018-19,1.0
87,Arsenal,2019-20,1.0
26,Aston Villa,2016-17,NaN


In [10]:
opponent_ids = team_ids.copy()
opponent_ids = opponent_ids.rename(columns = {'FIX_team' : 'FIX_opponent_team',
                                              'FIX_team_code' : 'FIX_opponent_code'}) 

In [11]:
opponent_ids.head()

,FIX_opponent_team,FIX_season,FIX_opponent_code
0,Arsenal,2016-17,1.0
1,Bournemouth,2016-17,2.0
2,Burnley,2016-17,3.0
3,Chelsea,2016-17,4.0
4,Crystal Palace,2016-17,5.0


In [12]:
fixtures_df = pd.merge(fixtures_df, team_ids, how = 'left',
         left_on = ['FIX_season','FIX_Team'] , right_on = ['FIX_season', 'FIX_team'])

fixtures_df = pd.merge(fixtures_df, opponent_ids, how = 'left',
         left_on = ['FIX_season','FIX_Opponent'] , right_on = ['FIX_season', 'FIX_opponent_team'])

In [13]:
fixtures_df.head()

,FIX_season,FIX_Team,FIX_GW,FIX_Fixture_date,FIX_Opponent,FIX_Home?,FIX_goals_marginal,FIX_goals_cumulative,FIX_opp_goals,FIX_points_marginal,FIX_points_cumulative,FIX_team,FIX_team_code,FIX_opponent_team,FIX_opponent_code
0,2019-20,Liverpool,1,2019-08-09,Norwich,1,4.0,4.0,1.0,3.0,3.0,Liverpool,10.0,Norwich,14.0
1,2019-20,Liverpool,2,2019-08-17,Southampton,0,2.0,6.0,1.0,3.0,6.0,Liverpool,10.0,Southampton,16.0
2,2019-20,Liverpool,3,2019-08-24,Arsenal,1,3.0,9.0,1.0,3.0,9.0,Liverpool,10.0,Arsenal,1.0
3,2019-20,Liverpool,4,2019-08-31,Burnley,0,3.0,12.0,0.0,3.0,12.0,Liverpool,10.0,Burnley,5.0
4,2019-20,Liverpool,5,2019-09-14,Newcastle,1,3.0,15.0,1.0,3.0,15.0,Liverpool,10.0,Newcastle,13.0


# Get player list for each year

## From GW data

In [14]:
GW_address = '/gws/*'
player_path =  dict(zip(season_names,[x + GW_address for x in season_folders]))

In [15]:
data_merge = []

for season in player_path:
    
    path = player_path[season]
    print(path)
    
    for fname in glob.glob(path):
            
        import_df = pd.read_csv(fname,encoding = "ISO-8859-1", parse_dates = ['kickoff_time'])

        if import_df.shape[0] > 0 :
            import_df['FIX_Fixture_date'] = import_df['kickoff_time'].dt.date 

            import_df['Season'] = season
            import_df['name'] = import_df['name'].str.replace('_\d+', '')
            
            data_merge.append(import_df)

player_gws = pd.concat(data_merge)
player_gws.head()

/Users/calumthompson/Documents/Fantasy_football/GitHub_data/data/2017-18/gws/*
/Users/calumthompson/Documents/Fantasy_football/GitHub_data/data/2016-17/gws/*
/Users/calumthompson/Documents/Fantasy_football/GitHub_data/data/2019-20/gws/*
/Users/calumthompson/Documents/Fantasy_football/GitHub_data/data/2018-19/gws/*


/Users/calumthompson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,FIX_Fixture_date,GW,Season,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,name,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards
0,2017-12-09,NaN,2017-18,0,38.0,0.0,0.0,0,22,1,8.0,28.0,11.3,0.0,0.0,443,0.0,0.0,159,1.0,0,0,3.0,8724.0,18.6,1.0,2017-12-09 12:30:00+00:00,09 Dec 12:30,0.0,0.0,90,Aaron_Cresswell,0.0,0.0,5,0,0.0,0,0,7.0,0,16,0,39148,1.0,0.0,0.0,0.0,1.0,0.0,5,-1314,819,2133,50,True,0.0,1
1,2017-12-10,NaN,2017-18,0,12.0,0.0,0.0,0,2,0,1.0,6.0,0.9,0.0,0.0,153,0.0,0.0,154,1.0,0,0,0.2,8434.0,1.0,0.0,2017-12-10 14:15:00+00:00,10 Dec 14:15,0.0,0.0,45,Aaron_Lennon,0.0,0.0,10,0,0.0,0,0,0.0,0,16,0,4864,0.0,0.0,0.0,1.0,1.0,0.0,1,14,313,299,54,False,0.0,0
2,2017-12-09,NaN,2017-18,0,82.0,0.0,0.0,0,14,1,0.0,68.0,33.5,1.0,0.0,172,0.0,0.0,153,0.0,0,0,7.0,8453.0,16.8,1.0,2017-12-09 15:00:00+00:00,09 Dec 15:00,0.0,0.0,90,Aaron_Mooy,0.0,3.0,3,0,0.0,0,0,8.0,0,16,0,329038,3.0,0.0,0.0,0.0,2.0,20.0,3,-15998,5326,21324,54,True,0.0,0
3,2017-12-09,NaN,2017-18,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,549,0.0,0.0,152,0.0,0,0,0.0,8830.0,0.0,0.0,2017-12-09 15:00:00+00:00,09 Dec 15:00,0.0,0.0,0,Aaron_Ramsdale,0.0,0.0,6,0,0.0,0,0,0.0,0,16,0,13253,0.0,0.0,0.0,2.0,2.0,0.0,0,-82,194,276,40,False,0.0,0
4,2017-12-10,NaN,2017-18,0,53.0,0.0,0.0,0,12,0,1.0,50.0,6.8,2.0,0.0,18,0.0,0.0,160,1.0,1,0,7.9,8299.0,14.2,0.0,2017-12-10 12:00:00+00:00,10 Dec 12:00,0.0,0.0,90,Aaron_Ramsey,0.0,0.0,14,0,0.0,0,0,3.0,0,16,0,576769,1.0,0.0,1.0,1.0,1.0,58.0,2,187251,203836,16585,72,False,0.0,0


## Add player positions
1 = GK 2 = DEF 3 = MID 4 = FWD

In [16]:
GW_address = '/players_raw.csv'
position_data =  dict(zip(season_names,[x + GW_address for x in season_folders]))

In [17]:
data_merge = []

for season in position_data:
    
    path = position_data[season]
    print(path)

    import_df = pd.read_csv(path,encoding = "ISO-8859-1", usecols = ['id','element_type'])
    import_df['Season'] = season

    data_merge.append(import_df)
    
positions_df = pd.concat(data_merge)

player_df = pd.merge(left = player_gws, right = positions_df , how = 'left',left_on = ['element','Season'], right_on = ['id', 'Season'])

player_df = player_df.rename(columns = {'element_type': 'position',
                                        'opponent_team' : 'FIX_opponent_code'})

player_df.head()

/Users/calumthompson/Documents/Fantasy_football/GitHub_data/data/2017-18/players_raw.csv
/Users/calumthompson/Documents/Fantasy_football/GitHub_data/data/2016-17/players_raw.csv
/Users/calumthompson/Documents/Fantasy_football/GitHub_data/data/2019-20/players_raw.csv
/Users/calumthompson/Documents/Fantasy_football/GitHub_data/data/2018-19/players_raw.csv


,FIX_Fixture_date,GW,Season,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id_x,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,name,offside,open_play_crosses,FIX_opponent_code,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,position,id_y
0,2017-12-09,NaN,2017-18,0,38.0,0.0,0.0,0,22,1,8.0,28.0,11.3,0.0,0.0,443,0.0,0.0,159,1.0,0,0,3.0,8724.0,18.6,1.0,2017-12-09 12:30:00+00:00,09 Dec 12:30,0.0,0.0,90,Aaron_Cresswell,0.0,0.0,5,0,0.0,0,0,7.0,0,16,0,39148,1.0,0.0,0.0,0.0,1.0,0.0,5,-1314,819,2133,50,True,0.0,1,2,443
1,2017-12-10,NaN,2017-18,0,12.0,0.0,0.0,0,2,0,1.0,6.0,0.9,0.0,0.0,153,0.0,0.0,154,1.0,0,0,0.2,8434.0,1.0,0.0,2017-12-10 14:15:00+00:00,10 Dec 14:15,0.0,0.0,45,Aaron_Lennon,0.0,0.0,10,0,0.0,0,0,0.0,0,16,0,4864,0.0,0.0,0.0,1.0,1.0,0.0,1,14,313,299,54,False,0.0,0,3,153
2,2017-12-09,NaN,2017-18,0,82.0,0.0,0.0,0,14,1,0.0,68.0,33.5,1.0,0.0,172,0.0,0.0,153,0.0,0,0,7.0,8453.0,16.8,1.0,2017-12-09 15:00:00+00:00,09 Dec 15:00,0.0,0.0,90,Aaron_Mooy,0.0,3.0,3,0,0.0,0,0,8.0,0,16,0,329038,3.0,0.0,0.0,0.0,2.0,20.0,3,-15998,5326,21324,54,True,0.0,0,3,172
3,2017-12-09,NaN,2017-18,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,549,0.0,0.0,152,0.0,0,0,0.0,8830.0,0.0,0.0,2017-12-09 15:00:00+00:00,09 Dec 15:00,0.0,0.0,0,Aaron_Ramsdale,0.0,0.0,6,0,0.0,0,0,0.0,0,16,0,13253,0.0,0.0,0.0,2.0,2.0,0.0,0,-82,194,276,40,False,0.0,0,1,549
4,2017-12-10,NaN,2017-18,0,53.0,0.0,0.0,0,12,0,1.0,50.0,6.8,2.0,0.0,18,0.0,0.0,160,1.0,1,0,7.9,8299.0,14.2,0.0,2017-12-10 12:00:00+00:00,10 Dec 12:00,0.0,0.0,90,Aaron_Ramsey,0.0,0.0,14,0,0.0,0,0,3.0,0,16,0,576769,1.0,0.0,1.0,1.0,1.0,58.0,2,187251,203836,16585,72,False,0.0,0,3,18


# Team market values

Using Soloman's webscraper

In [18]:
# teams_mv = pd.concat([build_season_mv('1617', 10, range(12,32)),
#                       build_season_mv('1718', 10, range(12,32)),
#                       build_season_mv('1819', 10, range(12,32)),
#                       build_season_mv('1920', 10, range(12,32))], 
#                      axis=0, ignore_index=True, sort=False)

# Merge

In [19]:
dataset_df = pd.merge(fixtures_df, player_df
                    , on = ['FIX_opponent_code','FIX_Fixture_date'], how = 'left')

dataset_df['FIX_Fixture_date'] = pd.to_datetime(dataset_df['FIX_Fixture_date'])

# Player value today

In [20]:
value_df = pd.DataFrame(dataset_df.sort_values(['FIX_Fixture_date']).groupby('name').last()['value']).reset_index()

value_df = value_df.rename(columns = {'value' : 'cost_today'})

In [21]:
dataset_df = pd.merge(dataset_df, value_df, how = 'left', on = 'name')

# Export

In [22]:
dataset_df.loc[dataset_df['FIX_Fixture_date'] < dt.today()].to_csv('Data/historic.csv', index = False)
dataset_df.loc[dataset_df['FIX_Fixture_date'] > dt.today()].to_csv('Data/forecast.csv', index = False)

In [23]:
dataset_df.loc[dataset_df['FIX_Fixture_date'] < dt.today()]['FIX_season'].unique()

array(['2019-20', '2018-19', '2017-18', '2016-17'], dtype=object)

In [24]:
dataset_df

,FIX_season,FIX_Team,FIX_GW,FIX_Fixture_date,FIX_Opponent,FIX_Home?,FIX_goals_marginal,FIX_goals_cumulative,FIX_opp_goals,FIX_points_marginal,FIX_points_cumulative,FIX_team,FIX_team_code,FIX_opponent_team,FIX_opponent_code,GW,Season,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id_x,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,name,offside,open_play_crosses,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,position,id_y,cost_today
0,2019-20,Liverpool,1,2019-08-09,Norwich,1,4.0,4.0,1.0,3.0,3.0,Liverpool,10.0,Norwich,14.0,NaN,2019-20,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,195.0,NaN,NaN,1.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,2019-08-09 19:00:00+00:00,NaN,NaN,NaN,0.0,Adam_Lallana,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,1.0,0.0,7143.0,NaN,NaN,NaN,1.0,4.0,0.0,0.0,0.0,0.0,0.0,60.0,True,NaN,0.0,3.0,195.0,57.0
1,2019-20,Liverpool,1,2019-08-09,Norwich,1,4.0,4.0,1.0,3.0,3.0,Liverpool,10.0,Norwich,14.0,NaN,2019-20,0.0,NaN,NaN,NaN,0.0,7.0,0.0,NaN,NaN,0.0,NaN,NaN,526.0,NaN,NaN,1.0,NaN,1.0,0.0,1.1,NaN,10.6,NaN,2019-08-09 19:00:00+00:00,NaN,NaN,NaN,51.0,AdriÃ¡n_San Miguel del Castillo,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,1.0,2.0,0.0,NaN,NaN,NaN,1.0,4.0,0.0,1.0,0.0,0.0,0.0,45.0,True,NaN,0.0,1.0,526.0,40.0
2,2019-20,Liverpool,1,2019-08-09,Norwich,1,4.0,4.0,1.0,3.0,3.0,Liverpool,10.0,Norwich,14.0,NaN,2019-20,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,193.0,NaN,NaN,1.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,2019-08-09 19:00:00+00:00,NaN,NaN,NaN,0.0,Alex_Oxlade-Chamberlain,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,1.0,0.0,28598.0,NaN,NaN,NaN,1.0,4.0,0.0,0.0,0.0,0.0,0.0,65.0,True,NaN,0.0,3.0,193.0,62.0
3,2019-20,Liverpool,1,2019-08-09,Norwich,1,4.0,4.0,1.0,3.0,3.0,Liverpool,10.0,Norwich,14.0,NaN,2019-20,0.0,NaN,NaN,NaN,0.0,7.0,0.0,NaN,NaN,0.0,NaN,NaN,189.0,NaN,NaN,1.0,NaN,0.0,0.0,0.6,NaN,5.6,NaN,2019-08-09 19:00:00+00:00,NaN,NaN,NaN,38.0,Alisson_Ramses Becker,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,1.0,2.0,1769075.0,NaN,NaN,NaN,1.0,4.0,0.0,1.0,0.0,0.0,0.0,60.0,True,NaN,0.0,1.0,189.0,62.0
4,2019-20,Liverpool,1,2019-08-09,Norwich,1,4.0,4.0,1.0,3.0,3.0,Liverpool,10.0,Norwich,14.0,NaN,2019-20,0.0,NaN,NaN,NaN,0.0,15.0,0.0,NaN,NaN,24.7,NaN,NaN,181.0,NaN,NaN,1.0,NaN,1.0,0.0,4.4,NaN,12.4,NaN,2019-08-09 19:00:00+00:00,NaN,NaN,NaN,90.0,Andrew_Robertson,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,1.0,0.0,1889240.0,NaN,NaN,NaN,1.0,4.0,7.0,2.0,0.0,0.0,0.0,70.0,True,NaN,0.0,2.0,181.0,70.0
5,2019-20,Liverpool,1,2019-08-09,Norwich,1,4.0,4.0,1.0,3.0,3.0,Liverpool,10.0,Norwich,14.0,NaN,2019-20,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,186.0,NaN,NaN,1.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,2019-08-09 19:00:00+00:00,NaN,NaN,NaN,0.0,Dejan_Lovren,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,1.0,0.0,18316.0,NaN,NaN,NaN,1.0,4.0,0.0,0.0,0.0,0.0,0.0,55.0,True,NaN,0.0,2.0,186.0,53.0
6,2019-20,Liverpool,1,2019-08-09,Norwich,1,4.0,4.0,1.0,3.0,3.0,Liverpool,10.0,Norwich,14.0,NaN,2019-20,1.0,NaN,NaN,NaN,3.0,46.0,0.0,NaN,NaN,26.4,NaN,NaN,188.0,NaN,NaN,1.0,NaN,1.0,1.0,9.3,NaN,45.4,NaN,2019-08-09 19:00:00+00:00,NaN,NaN,NaN,73.0,Divock_Origi,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,1.0,0.0,165505.0,NaN,NaN,NaN,1.0,4.0,21.0,12.0,0.0,0.0,0.0,55.0,True,NaN,0.0,4.0,188.0,51.0
7,2019-20,Liverpool,1,2019-08-09,Norwich,1,4.0,4.0,1.0,3.0,3.0,Liverpool,10.0,Norwich,14.0,NaN,2019-20,0.0,NaN,NaN,NaN,0.0,18.0,0.0,NaN,NaN,11.8,NaN,NaN,197.0,NaN,NaN,1.0,NaN,1.0,0.0,4.4,NaN,25.6,NaN,2019-08-09 19:00:00+00:00,NaN,NaN,NaN,90.0,Fabio Henrique_Tavares,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,1.0,0.0,49706.0,NaN,NaN,NaN,1.0,4.0,7.0,2.0,0.0,0.0,0.0,55.0,True,NaN,0.0,3.0,197.0,54.0
8,2019-20,Liverpool,